In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import os

% pylab inline

Populating the interactive namespace from numpy and matplotlib


# Read in data and Visualize

In [23]:
# read in data sets
student = pd.read_csv('data/hsls_09_student_v3_0.csv')
school = pd.read_csv('data/HSLS_09_SCHOOL_v1_0.csv')

In [24]:
school.head()

,Sch_ID,X1NCESID,W1SCHOOL,STRAT_ID,PSU,X1CONTROL,X1LOCALE,X1REGION,X1CENDIV,X1STATESAMPL,...,W1SCHOOL191,W1SCHOOL192,W1SCHOOL193,W1SCHOOL194,W1SCHOOL195,W1SCHOOL196,W1SCHOOL197,W1SCHOOL198,W1SCHOOL199,W1SCHOOL200
0,1001,-5,2.331907,-5,-5,1,1,3,-5,-5,...,0.000000,0.000000,3.601229,0.00000,5.203120,0.000000,0.000000,0.000000,0.000000,0.000000
1,1002,-5,4.615802,-5,-5,1,2,3,-5,-5,...,0.000000,5.117174,5.931073,0.00000,0.000000,0.000000,5.537744,6.027139,0.000000,0.000000
2,1003,-5,8.810384,-5,-5,1,1,3,-5,-5,...,0.000000,17.768776,13.514905,17.79419,20.618902,16.673944,24.577378,0.000000,0.000000,0.000000
3,1004,-5,8.845971,-5,-5,2,2,2,-5,-5,...,0.000000,9.884404,0.000000,0.00000,0.000000,6.866897,25.398480,6.175511,21.028035,5.663044
4,1005,-5,10.807970,-5,-5,1,2,3,-5,-5,...,21.067261,12.796792,11.042852,0.00000,22.656201,16.703235,0.000000,20.871432,22.551557,0.000000


In [25]:
student.head()

,STU_ID,SCH_ID,X1NCESID,X2NCESID,STRAT_ID,PSU,X2UNIV1,X2UNIV2A,X2UNIV2B,X3UNIV1,...,W3W2STUTR191,W3W2STUTR192,W3W2STUTR193,W3W2STUTR194,W3W2STUTR195,W3W2STUTR196,W3W2STUTR197,W3W2STUTR198,W3W2STUTR199,W3W2STUTR200
0,10001,-5,-5,-5,-5,-5,11,1,1,1111,...,0.000000,681.990359,699.375850,0.000000,970.185221,0.000000,0.000000,865.089553,0.000000,869.152511
1,10002,-5,-5,-5,-5,-5,11,1,1,1111,...,288.072989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,297.689074,335.434801,0.000000
2,10003,-5,-5,-5,-5,-5,11,1,1,1111,...,331.062262,0.000000,0.000000,208.192071,320.617580,456.194115,213.006544,289.194529,353.621803,0.000000
3,10004,-5,-5,-5,-5,-5,10,1,7,1001,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10005,-5,-5,-5,-5,-5,11,1,1,1111,...,0.000000,308.873275,275.385607,296.583225,0.000000,284.432874,382.436956,0.000000,262.186372,322.338089


# Clean Data
check dictionaries for details:
* https://nces.ed.gov/surveys/hsls09/pdf/2011328_1.pdf
* https://nces.ed.gov/surveys/hsls09/pdf/2011328_2.pdf
* https://nces.ed.gov/pubsearch/pubsinfo.asp?pubid=2014361

 
 # To Do
 * Decide how to deal with various categorical variables??? Dummys? Encoded? We will have to go through each feature individually unfortunately. Maybe divide and conquer. Some are ordinal while others have no order and may need to be converted to dummys. 
 * Clean up double variables. For instance Race has X1Race (ordinal) as well as X1Black, X1White etc. (binary) we need to delete one of these. In the case of race we cannot have ordinal values, they will need to be dummys, but this may be different for something like parent education level (although this could be contenious?). Maybe we can make a spreadsheet with all features and categorise what to do with them.
 * Ckean up double variable between Questionaaires. Parent questionaire asks parent race etc. Student is also asked these questions.

#### Restricted Data
All restricted data sets have -5 data in them.

We may need to decide what to do with this data should it become available.

In [26]:
# Check empty columns =-5 - this takes long to run
colcounts = student[student==-5].count()
colcounts[(colcounts > 0) & (colcounts != 23503)].sum()

0

All rows with some -5 values are completely empty so we can remove

In [27]:
# List all restricted values in list
restrictedcols = student.columns[colcounts == 23503] 

restrictedcols

Index([u'SCH_ID', u'X1NCESID', u'X2NCESID', u'STRAT_ID', u'PSU', u'X1ASIAN',
       u'X1PACISLE', u'X1AMINDIAN', u'X1HISPTYPE', u'X1ASIANTYPE',
       ...
       u'C2UPBOUND', u'C2CLGEXAMINFO', u'C2CLGINFO', u'C2NUMAPEXAM',
       u'C2NUMAP3PLUS', u'C2NUMIBEXAM', u'C2NUMIB4PLUS', u'C2NUMAPANDIB',
       u'X1NATIVEL_IM', u'X2NATIVEL_IM'],
      dtype='object', length=789)

#### Missing Data

* -9 = Missing Data
* -8 = Unit non-response/component not applicable
* -7 = Item legitimate skip/NA 

We need to decide when we want to delete this data or not. Often this data missing may mean something, for instacne a parent may intentionally not fill in their job, this does not mean this is not a valid data point. I believe we should keep it in. 

For instance a child may not have a second parent hence they have a -8 response for all questions pertaining to parent 2. This is still useful info surely? Maybe create dummy for no second parent?


In [76]:
student_[student_==-9].count()

STU_ID                0
X2UNIV1               0
X2UNIV2A              0
X2UNIV2B              0
X3UNIV1               0
X1SEX                 6
X1RACE             1006
X1HISPANIC         1006
X1BLACK             565
X1DUALLANG           25
X1STDOB             186
X1TXMTH               0
X1TXMSEM              0
X1TXMSCR              0
X1TXMTSCOR            0
X1TXMQUINT            0
X1TXMPROF1            0
X1TXMPROF2            0
X1TXMPROF3            0
X1TXMPROF4            0
X1TXMPROF5            0
X1MACC                0
X1PARRESP             0
X1P1RELATION          0
X1PAR1EDU             4
X1PAR1EMP            27
X1PAR1OCC2           14
X1PAR1OCC_STEM1      14
X1PAR1RACE          893
X1P2RELATION          0
                   ... 
X2P2RELAT_IM          0
X2PAR1EDU_IM          0
X2PAR2EDU_IM          0
X2PAREDU_IM           0
X2PARPATT_IM          0
X2PAR1EMP_IM          0
X2PAR2EMP_IM          0
X2PAR1OCC_IM          0
X2PAR2OCC_IM          0
X2MOMREL_IM           0
X2MOMEDU_IM     

In [75]:
student_[student_==-8].count()

STU_ID                0
X2UNIV1               0
X2UNIV2A              0
X2UNIV2B              0
X3UNIV1               0
X1SEX                 0
X1RACE                0
X1HISPANIC            0
X1BLACK               0
X1DUALLANG         2059
X1STDOB               0
X1TXMTH            2059
X1TXMSEM           2059
X1TXMSCR           2059
X1TXMTSCOR         2059
X1TXMQUINT         2059
X1TXMPROF1         2059
X1TXMPROF2         2059
X1TXMPROF3         2059
X1TXMPROF4         2059
X1TXMPROF5         2059
X1MACC             2722
X1PARRESP          6715
X1P1RELATION       6715
X1PAR1EDU          6715
X1PAR1EMP          6715
X1PAR1OCC2         6715
X1PAR1OCC_STEM1    6715
X1PAR1RACE         6715
X1P2RELATION       6715
                   ... 
X2P2RELAT_IM          0
X2PAR1EDU_IM          0
X2PAR2EDU_IM          0
X2PAREDU_IM           0
X2PARPATT_IM          0
X2PAR1EMP_IM          0
X2PAR2EMP_IM          0
X2PAR1OCC_IM          0
X2PAR2OCC_IM          0
X2MOMREL_IM           0
X2MOMEDU_IM     

#### Actual Cleaning step

In [28]:
# Drop all restricted data for this analysis
student_= student.drop(restrictedcols, axis=1)

# Remove all weighting variables
student_ = student_[student_.columns[~student_.columns.str.contains('W')]]

### Add other data cleaning steps in here


#### Remove Features that only provide information after the fact
Depending on the question certain features may not be relevant. For instance if you got your GDE because you did not dropped out. We would not want to include that as a feature for prediction of dropout

In [85]:
### Code for cleaning exp-post variables depednig on question

# Create data set without imputed data

In [46]:
# Determine all rows with imputed or altered values in them
student_['Imputed?'] = student_[student_.columns[student_.columns.str.contains('_IM')]].T.sum()

# Calculate percentage imputed data sets
print(str(100.0 * (student_['Imputed?']>0).sum()/len(student_)) + '% of the data was imputed ')

77.7730502489% of the data was imputed 


It seems like most of the data had some of it imputed, maybe we can look at how much was imputed? If imputed less than 10 data points or some other random variable?
#### Create data set with no imputed data

In [62]:
# Drop rows with imputed data
student_noimputed = student_[student_['Imputed?']==0]
###############################################################################

# remove impudation columns
# _IM - whetehr or not imputed
# _U value for determing feature through imputing
student_noimputed = (student_noimputed[student_noimputed.columns[~student_noimputed.columns.str.contains('_IM|_U')]])
otherimputecols = ['X1TXMTH1','X1TXMTH2','X1TXMTH3','X1TXMTH4',
                   'X1TXMTH5','X1TXMSEM1','X1TXMSEM2','X1TXMSEM3',
                   'X1TXMSEM4','X1TXMSEM5','X1SES1','X1SES2','X1SES3',
                   'X1SES4','X1SES5','X2TXMTH1','X2TXMTH2','X2TXMTH3',
                   'X2TXMTH4','X2TXMTH5','X2TXMSEM1','X2TXMSEM2',
                   'X2TXMSEM3','X2TXMSEM4','X2TXMSEM5','X2SES1',
                   'X2SES2','X2SES3','X2SES4','X2SES5','Imputed?']
student_noimputed.drop(otherimputecols, axis=1, inplace=True)

# Visualise data
student_noimputed.head()

,STU_ID,X2UNIV1,X2UNIV2A,X2UNIV2B,X3UNIV1,X1SEX,X1RACE,X1HISPANIC,X1BLACK,X1DUALLANG,...,C2AVGACTMATH,C2AVGACTREAD,C2AVGACTSCI,C2AVGACTCOMP,C2AVGACTNONE,C2STUSURVEY,C2DATABASE,C2FBREMEDIAL,C2FB1STYR,C2FBGRAD
0,10001,11,1,1,1111,1,8,0,0,1,...,23,22,22,22,0,0,0,1,1,1
2,10003,11,1,1,1111,2,3,0,1,1,...,-9,-9,-9,-9,0,1,0,1,1,1
3,10004,10,1,7,1001,2,8,0,0,1,...,-9,-9,-9,-9,1,1,0,2,1,1
4,10005,11,1,1,1111,1,8,0,0,1,...,-9,-9,-9,-9,0,1,0,-9,-9,-9
9,10010,11,1,1,1111,2,8,0,0,3,...,-9,-9,-9,24,0,0,-9,2,1,1


In [96]:
student_noimputed[student_noimputed.columns[student_noimputed.columns.str.contains('S3')]].head()

,X3TCRSES3,S3HSCRED,S3HSCREDTYPE,S3HSCREDYR,S3ENROLLHS13,S3HSPLAN,S3HSCREDPLAN,S3HSCOMPYR,S3LASTHSYR,S3LASTHS,...,S3APPRENTSHIP,S3JOBSTARTMO,S3JOBSTARTYR,S3OTHJOB,S3OTHJOBHRS,S3OTHJOBFT,S3NOV1JOBPLAN,S3NOV1JOB2,S3NOV1JOB_STEM1,S3FALLHS
0,1,1,1,2013,-7,-7,-7,-7,-7,4,...,1,6,2013,0,-7,-7,-7,-7,-7,-7
2,0,1,1,2013,-7,-7,-7,-7,-7,4,...,-7,-7,-7,-7,-7,-7,-7,43,0,-7
3,0,-8,-8,-8,-8,-8,-8,-8,-8,-8,...,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8
4,1,1,1,2013,-7,-7,-7,-7,-7,4,...,-7,-7,-7,-7,-7,-7,-7,99,9,-7
9,1,1,1,2013,-7,-7,-7,-7,-7,4,...,1,5,2012,0,-7,-7,-7,-7,-7,-7
